# ASSIGNMENT

Use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

### Building Ollama model

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [6]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can generate high-quality content such as articles, social media posts, and product descriptions, saving time and resources for content creation teams.
2. **Marketing Automation**: Generative AI can be used to create personalized marketing campaigns, product recommendations, and customer segmentation strategies, enhancing the overall customer experience.
3. **Product Design**: Generative AI can help designers create new products, such as furniture, clothing, or electronics, by generating 3D models, prototypes, and even production-ready designs.
4. **Chatbots and Virtual Assistants**: Generative AI-powered chatbots can provide personalized customer support, answer frequently asked questions, and even offer product recommendations.
5. **Image and Video Generation**: Generative AI can create realistic images and videos for various applications such as

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [7]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI (Generative Artificial Intelligence) has numerous business applications across various industries, including:

1. **Content Creation**: AI-powered tools can generate high-quality content such as articles, blog posts, product descriptions, social media posts, and even entire books.
2. **Marketing Automation**: Generative AI can help automate marketing campaigns by generating personalized messages, subject lines, and email copy that resonate with customers.
3. **Customer Service Chatbots**: AI-powered chatbots can use generative models to generate responses to customer inquiries, providing 24/7 support and improving customer satisfaction.
4. **Image and Video Generation**: Generative AI can create realistic images and videos for various applications such as advertising, product visualization, and entertainment.
5. **Product Design and Development**: AI-powered tools can generate design concepts, prototypes, and even entire product lines, reducing the need for extensive desi

## Alternative approach - using OpenAI python library to connect to Ollama

In [8]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI, also known as generative models or generative algorithms, has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Companies can use generative AI to create high-quality content such as:
	* Articles, blog posts, and social media posts.
	* Product descriptions, product pitches, and marketing copy.
	* Music, videos, and podcasts.
2. **Image and Video Synthesis**: Generative AI can generate realistic images and videos for:
	* Advertising and marketing campaigns (e.g., logo creation, branding materials).
	* Virtual events, conferences, and product presentations.
	* E-commerce and online shopping experiences (e.g., lifestyle product images, fashion shows).
3. **Design Assistance**: Generative AI can aid designers in creating innovative designs by generating:
	* Architectural designs, blueprints, and 3D models.
	* Fashion designs, textiles, and accessories.
	* Product design concepts, prototypes, and manufacturing specif

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [9]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 402 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.7 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.5 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.9 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 3.3 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕             

In [10]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Okay, so I need to explain the key terms related to Large Language Models (LLMs), focusing on neural networks, attention mechanisms, and transformers. Hmm, where do I start? Well, I know that LLMs are big language models, which means they can generate text. But I'm not exactly sure how they're structured. 

Alright, first the neural network part. From what I remember, neural networks are a set of algorithms inspired by the human brain's structure and function. They process data through layers of connected nodes or neurons. Each layer processes the information step by step until the output gets to the final prediction layer. But how does this relate to language models? Oh right, they're actually trained using a technique called fine-tuning where they're built based on task-specific data (like texts of millions of books) and then fine-tuned with new text.

Next up is attention mechanisms. I think attention is about modeling the relevance or importance of different parts in the in

# NOW, extract the data from website and build the Ollama model for get the summary

In [11]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

#### scrap the website

In [13]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [14]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [15]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [16]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [17]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [18]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

#### TEST OpenAI library used to connect Ollama

In [19]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Ugh, really? You need me to tell you that the answer is... *dramatic pause* ...4. Congratulations, I hope it was painful for you not knowing that already. Next thing you know, you'll be asking me what the sun looks like.


In [20]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [21]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

#### Build function for OpenAI library used to connect Ollama

In [24]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:


def summarize(url):
    website = Website(url)
    
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [25]:
summarize("https://edwarddonner.com")

'### Website Summary\n\n**Introduction**\nThe website is owned by Edward Donner, a co-founder and CTO of Nebula.io. It appears to be an online platform that leverages Large Language Models (LLMs) in various applications.\n\n**Content Highlights**\n- **Connecting my courses**: An announcement about upcoming courses on becoming an LLM expert and leader.\n- **2025 AI Executive Briefing**: Information about a forthcoming event focused on executive AI briefings.\n- **Courses and training programs**: Multiple announcements for different LLM-related courses, including "The Complete Agentic AI Engineering Course" and the "LLM Workshop – Hands-on with Agents".\n- **Personal updates and anecdotes**: Edward shares his personal interests in DJing, electronic music production, and his affinity for Hacker News.\n\n### No notable news or announcements were found.'

In [26]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [27]:
display_summary("https://edwarddonner.com")

### Website Summary

#### Overview
The website belongs to Edward Donner, the co-founder and CTO of Nebula.io, an AI startup applying AI to help people discover their potential. The website is primarily a personal blog or profile.

#### Content Highlights

*   **Personal Story**: A brief introduction about Edward's interests in writing code, experimenting with LLMs, and his experience as a founder and CEO of AI startups.
*   **Nebula.io Updates**
    *   Connecting my courses - become an LLM expert and leader (May 28, 2025)
    *   2025 AI Executive Briefing (April 21, 2025)
    *   The Complete Agentic AI Engineering Course (January 23, 2025)

No news or announcements are summarised in the provided content.

*   **LLM Arena and Competitions**: The "Outsmart" section describes an arena where LLMs compete against each other.
*   **Get in Touch**: Edward's contact information is provided.

In [29]:
display_summary("https://cnn.com")

**Summary**

The **Breaking News, Latest News and Videos | CNN** website provides up-to-date news coverage on various topics, including:

### US Politics

* Breaking news: Trump's 'breathing space'
* Iran-Iraq conflict
* Israeli military strikes in Iran
* Trump's decision on US involvement in Israel-Iran conflict

### World Politics

* US-CNBC trade negotiation
* China sends aircraft carriers to Pacific
* North Korea COVID-19 report denied
* Trump promised peacemaker presidency

### Business and Finance

* Markets forecast: Wall Street reactions
* Tech industry news: AI, Amazon, Uber
* Healthcare updates

### Health and Wellness

* Heatstroke prevention
* Microplastic pollution in US coastal waters
* Fitness advice

### Technology

* AI warnings and job security
* Facebook's plans to launch AI safety tools

### News and Features

* 'Dragon Man' DNA revelation
* NASA prepares for massive Mars sample return mission
* US, China, Israel, Russia and India among countries affected by climate change.

**Global Travel**

The world's best airline, airline, or flight destinations and airlines.